In [1]:
!pip install datasets transformers google-cloud-bigquery vertexai pandas tqdm


In [2]:
# ========================================
# Step 1: 下載、翻譯、存入 BigQuery
# 資料集：MakTek Customer Support FAQs
# 目標：willy-poc.chatbot_agent (新表)
# ========================================

import pandas as pd
import vertexai
from vertexai.generative_models import GenerativeModel, GenerationConfig
from google.cloud import bigquery
from datasets import load_dataset
from tqdm import tqdm
import time

# === 環境設定 ===
PROJECT_ID = "willy-poc"
DATASET = "chatbot_agent"
TABLE_NAME = "customer_support_faqs"
BQ_LOCATION = "US"

# 初始化 GCP 服務
vertexai.init(project=PROJECT_ID, location="us-central1")
bq = bigquery.Client(project=PROJECT_ID, location=BQ_LOCATION)

# 初始化 Gemini 翻譯模型
model = GenerativeModel("gemini-2.5-flash")
gen_config = GenerationConfig(
    temperature=0.2,  # 低溫度讓翻譯更穩定
    max_output_tokens=2048,
)

print("✅ 環境初始化完成")

# ========================================
# Step 2: 下載 MakTek 資料集
# ========================================

print("\n📥 開始下載 MakTek Customer Support FAQs...")

try:
    # 從 Hugging Face 下載
    dataset = load_dataset("MakTek/Customer_support_faqs_dataset", split="train")
    print(f"✅ 下載完成！共 {len(dataset)} 筆資料")
    
    # 轉換成 pandas DataFrame
    df = dataset.to_pandas()
    print("\n📊 資料範例：")
    print(df.head(3))
    
except Exception as e:
    print(f"❌ 下載失敗：{e}")
    print("\n💡 如果下載失敗，請先安裝 datasets 套件：")
    print("   pip install datasets")
    raise



/opt/conda/lib/python3.10/site-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


✅ 環境初始化完成

📥 開始下載 MakTek Customer Support FAQs...
✅ 下載完成！共 200 筆資料

📊 資料範例：
                              question  \
0         How can I create an account?   
1  What payment methods do you accept?   
2            How can I track my order?   

                                              answer  
0  To create an account, click on the 'Sign Up' b...  
1  We accept major credit cards, debit cards, and...  
2  You can track your order by logging into your ...  


In [3]:
# ========================================
# Step 3: 使用 Gemini 翻譯成繁體中文
# ========================================

print("\n🌏 開始翻譯成台灣繁體中文...")

def translate_to_zh_tw(text: str, retry: int = 3) -> str:
    """使用 Gemini 翻譯成台灣繁體中文（電商客服語境）"""
    
    prompt = f"""請將以下英文客服對話翻譯成台灣繁體中文。

要求：
1. 使用台灣常見的電商客服用語
2. 保持專業、友善的語氣
3. 只輸出翻譯結果，不要加任何說明
4. 如果有專有名詞（如 email, account），保留英文或用常見譯法

英文：
{text}

繁體中文："""
    
    for attempt in range(retry):
        try:
            response = model.generate_content(
                prompt,
                generation_config=gen_config
            )
            result = response.text.strip()
            
            # 清理可能的前綴
            result = result.replace("繁體中文：", "").strip()
            result = result.strip('"').strip("'")
            
            return result
        
        except Exception as e:
            print(f"⚠️  翻譯失敗 (嘗試 {attempt + 1}/{retry})：{e}")
            if attempt < retry - 1:
                time.sleep(2)  # 等待 2 秒後重試
            else:
                return text  # 最後一次失敗就返回原文

# 翻譯問題和答案
questions_zh = []
answers_zh = []

for idx, row in tqdm(df.iterrows(), total=len(df), desc="翻譯進度"):
    # 翻譯問題
    q_zh = translate_to_zh_tw(row['question'])
    questions_zh.append(q_zh)
    
    # 翻譯答案
    a_zh = translate_to_zh_tw(row['answer'])
    answers_zh.append(a_zh)
    
    # 避免 API 限流，每 10 筆休息 1 秒
    if (idx + 1) % 10 == 0:
        time.sleep(1)

# 加入翻譯結果到 DataFrame
df['question_zh'] = questions_zh
df['answer_zh'] = answers_zh

print("✅ 翻譯完成！")
print("\n📊 翻譯範例：")
print(f"\n英文問題：{df.iloc[0]['question']}")
print(f"中文問題：{df.iloc[0]['question_zh']}")
print(f"\n英文答案：{df.iloc[0]['answer'][:100]}...")
print(f"中文答案：{df.iloc[0]['answer_zh'][:100]}...")

# ========================================
# Step 4: 整理並存入 BigQuery
# ========================================

print(f"\n💾 開始存入 BigQuery：{PROJECT_ID}.{DATASET}.{TABLE_NAME}")

# 整理欄位（建立標準化結構）
df_final = pd.DataFrame({
    'id': range(1, len(df) + 1),  # 自動編號
    'question_en': df['question'],
    'answer_en': df['answer'],
    'question_zh': df['question_zh'],
    'answer_zh': df['answer_zh'],
    'lang': 'zh-TW',  # 標記為繁體中文
    'source': 'MakTek',  # 資料來源
    'category': 'General',  # 可以後續手動分類
})

# 預覽最終資料
print("\n📊 最終資料結構：")
print(df_final.head(3))
print(f"\n總共 {len(df_final)} 筆資料")

# 建立 Dataset（如果不存在）
try:
    bq.get_dataset(f"{PROJECT_ID}.{DATASET}")
    print(f"✅ Dataset {DATASET} 已存在")
except:
    dataset_obj = bigquery.Dataset(f"{PROJECT_ID}.{DATASET}")
    dataset_obj.location = BQ_LOCATION
    bq.create_dataset(dataset_obj)
    print(f"✅ 已建立 Dataset {DATASET}")

# 建立表格 Schema（定義欄位類型）
schema = [
    bigquery.SchemaField("id", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("question_en", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("answer_en", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("question_zh", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("answer_zh", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("lang", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("source", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("category", "STRING", mode="NULLABLE"),
]

# 設定寫入模式（覆蓋已存在的表）
job_config = bigquery.LoadJobConfig(
    schema=schema,
    write_disposition="WRITE_TRUNCATE",  # 覆蓋模式
)

# 寫入 BigQuery
table_id = f"{PROJECT_ID}.{DATASET}.{TABLE_NAME}"

try:
    job = bq.load_table_from_dataframe(
        df_final,
        table_id,
        job_config=job_config
    )
    job.result()  # 等待完成
    
    print(f"✅ 資料已成功寫入 BigQuery！")
    print(f"   表格：{table_id}")
    print(f"   筆數：{len(df_final)}")
    
except Exception as e:
    print(f"❌ 寫入失敗：{e}")
    raise

# ========================================
# Step 5: 驗證資料
# ========================================

print("\n🔍 驗證 BigQuery 資料...")

# 查詢前 5 筆
query = f"""
SELECT 
    id,
    question_zh,
    answer_zh,
    source
FROM `{table_id}`
ORDER BY id
LIMIT 5
"""

results = bq.query(query).result()

print("\n📊 BigQuery 中的資料範例：")
for row in results:
    print(f"\nID: {row.id}")
    print(f"問題：{row.question_zh}")
    print(f"答案：{row.answer_zh[:100]}...")
    print("-" * 80)

# 統計資訊
count_query = f"SELECT COUNT(*) as total FROM `{table_id}`"
total = list(bq.query(count_query).result())[0].total

print(f"\n✅ 驗證完成！BigQuery 中共有 {total} 筆資料")

# ========================================
# Step 6: 儲存本地備份（可選）
# ========================================

print("\n💾 儲存本地備份...")

# 存成 CSV（UTF-8 with BOM，Excel 可正常開啟）
local_csv = f"customer_support_faqs_zh_tw.csv"
df_final.to_csv(local_csv, index=False, encoding='utf-8-sig')
print(f"✅ 已儲存至：{local_csv}")

# 存成 JSON（方便程式讀取）
local_json = f"customer_support_faqs_zh_tw.json"
df_final.to_json(local_json, orient='records', force_ascii=False, indent=2)
print(f"✅ 已儲存至：{local_json}")

print("\n" + "="*80)
print("🎉 完成！資料已準備好，可以進入下一階段")
print("="*80)

print(f"""
📋 下一步：
1. 檢查 BigQuery 表格：{table_id}
2. 在原本的程式碼中，將資料來源指向這個新表格
3. 重新執行 embedding 生成流程

💡 表格結構：
- id: 編號
- question_zh / answer_zh: 繁體中文問答
- question_en / answer_en: 原始英文（保留參考）
- lang: 語言標記（zh-TW）
- source: 資料來源（MakTek）
- category: 分類（可後續調整）
""")



🌏 開始翻譯成台灣繁體中文...


翻譯進度: 100%|██████████| 200/200 [30:20<00:00,  9.10s/it]


✅ 翻譯完成！

📊 翻譯範例：

英文問題：How can I create an account?
中文問題：請問要怎麼註冊帳號？

英文答案：To create an account, click on the 'Sign Up' button on the top right corner of our website and follo...
中文答案：若要建立帳號，請點選我們網站右上角的「註冊」按鈕，並依照指示完成註冊程序。...

💾 開始存入 BigQuery：willy-poc.chatbot_agent.customer_support_faqs

📊 最終資料結構：
   id                          question_en  \
0   1         How can I create an account?   
1   2  What payment methods do you accept?   
2   3            How can I track my order?   

                                           answer_en       question_zh  \
0  To create an account, click on the 'Sign Up' b...        請問要怎麼註冊帳號？   
1  We accept major credit cards, debit cards, and...   請問貴公司接受哪些付款方式呢？   
2  You can track your order by logging into your ...  請問我該如何查詢我的訂單進度呢？   

                                  answer_zh   lang  source category  
0     若要建立帳號，請點選我們網站右上角的「註冊」按鈕，並依照指示完成註冊程序。  zh-TW  MakTek  General  
1       我們接受主要信用卡、金融卡以及 PayPal 作為線上訂單的付款方式。  zh-TW  MakTek  General  
2  您可以登入您的